# Loading Libraries

In [58]:
# Importing the core libraies
import numpy as np 
import pandas as pd 
from IPython.display import Markdown
from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
!pip install pycountry
import pycountry
from plotly.offline import init_notebook_mode, iplot 
import plotly.offline as py
import plotly.express as ex
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import matplotlib.pyplot as plt
py.init_notebook_mode(connected=True)
plt.style.use("seaborn-talk")
plt.rcParams['figure.figsize'] = 8, 5
plt.rcParams['image.cmap'] = 'viridis'
import folium
!pip install ipympl
%matplotlib notebook

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

pd.set_option('display.max_rows', None)
from math import sin, cos, sqrt, atan2, radians
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn import preprocessing
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings("ignore")

In [59]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Importing the dataset into python
#df = pd.read_csv('./covid19-global-forecasting-week-5/covid_19_data.csv',parse_dates=['ObservationDate'])
df = pd.read_csv('/content/drive/My Drive/Master Online Exams/Dataset/covid_19_data.csv',parse_dates=['ObservationDate'])
df.drop(['SNo','Last Update'],axis =1, inplace = True)
df['Active'] = df['Confirmed'] - (df['Recovered'] + df['Deaths'])

In [0]:
#week5_train = pd.read_csv('./covid19-global-forecasting-week/train.csv')
#week5_test = pd.read_csv('./covid19-global-forecasting-week-5/test.csv')
#dataset=pd.read_csv('./covid19-global-forecasting-week-5/covid_19_complete.csv',parse_dates=['Date'])
week5_train = pd.read_csv('/content/drive/My Drive/Master Online Exams/Dataset/train.csv')
week5_test = pd.read_csv('/content/drive/My Drive/Master Online Exams/Dataset/test.csv')
dataset=pd.read_csv('/content/drive/My Drive/Master Online Exams/Dataset/covid_19_complete.csv',parse_dates=['Date'])

# Data Cleaning

In [0]:
# Dataset Cleaning
# Creating a colunm for active cases
dataset['Active'] = dataset['Confirmed'] - dataset['Deaths'] - dataset['Recovered']
# Replacing the word Mainland China with china
dataset['Country/Region'] = dataset['Country/Region'].replace('Mainland China','China')
#Filling Missing values
dataset[['Province/State']] = dataset[['Province/State']].fillna('')
dataset[['Confirmed', 'Deaths', 'Recovered', 'Active']] = dataset[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)
# Datatypes
dataset['Recovered'] = dataset['Recovered'].astype(int)

In [0]:
# Creating Grouped dataset 
dataset_grouped = dataset.groupby(['Date','Country/Region'])['Confirmed','Deaths','Recovered','Active'].sum().reset_index()

# New cases 
temp = dataset_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp = temp.sum().diff().reset_index()
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan

#Renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered']
# Merging new values
# Dataset_grouped = pd.merge(dataset_grouped, temp, on=['Country/Region', 'Date'])
dataset_grouped = pd.merge(dataset_grouped, temp, on=['Country/Region', 'Date'])
# Filling na with 0
dataset_grouped = dataset_grouped.fillna(0)
# Fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
dataset_grouped[cols] = dataset_grouped[cols].astype('int')
dataset_grouped['New cases'] = dataset_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

In [0]:
# Country data grouping
country_wise = dataset_grouped[dataset_grouped['Date']==max(dataset_grouped['Date'])].reset_index(drop=True).drop('Date',axis=1)

#Grouped by Country
country_wise = country_wise.groupby('Country/Region')['Confirmed','Deaths','Recovered','Active','New cases'].sum().reset_index()

# Grouped per 100 cases
country_wise['Deaths / 100 Cases'] = round((country_wise['Deaths']/country_wise['Confirmed'])*100,2)
country_wise['Recovered / 100 cases'] = round((country_wise['Recovered']/country_wise['Confirmed'])*100,2)
country_wise['Deaths / 100 Recovered'] = round((country_wise['Deaths']/country_wise['Recovered'])*100,2)
cols= ['Deaths / 100 Cases','Recovered / 100 cases','Deaths / 100 Recovered']
country_wise[cols] = country_wise[cols].fillna(0)

# Grouping by Time Values
today = dataset_grouped[dataset_grouped['Date']==max(dataset_grouped['Date'])].reset_index(drop=True).drop('Date', axis=1)[['Country/Region', 'Confirmed']]
last_week = dataset_grouped[dataset_grouped['Date']==max(dataset_grouped['Date'])-timedelta(days=7)].reset_index(drop=True).drop('Date', axis=1)[['Country/Region', 'Confirmed']]
temp = pd.merge(today, last_week, on='Country/Region', suffixes=(' today', ' last week'))
#temp = temp[['Country/Region', 'Confirmed last week']]
temp['1 week change'] = temp['Confirmed today'] - temp['Confirmed last week']
temp = temp[['Country/Region', 'Confirmed last week', '1 week change']]
country_wise = pd.merge(country_wise, temp, on='Country/Region')
country_wise['1 week % increase'] = round(country_wise['1 week change']/country_wise['Confirmed last week']*100, 2)

In [0]:
day_wise = dataset_grouped.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases'].sum().reset_index()
# number cases per 100 cases
day_wise['Deaths / 100 Cases'] = round((day_wise['Deaths']/day_wise['Confirmed'])*100, 2)
day_wise['Recovered / 100 Cases'] = round((day_wise['Recovered']/day_wise['Confirmed'])*100, 2)
day_wise['Deaths / 100 Recovered'] = round((day_wise['Deaths']/day_wise['Recovered'])*100, 2)
# no. of countries
day_wise['No. of countries'] = dataset_grouped[dataset_grouped['Confirmed']!=0].groupby('Date')['Country/Region'].unique().apply(len).values
# fillna by 0
cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
day_wise[cols] = day_wise[cols].fillna(0)

In [0]:
week5_train = week5_train.drop(columns = ['County' , 'Province_State'])
week5_test = week5_test.drop(columns = ['County' , 'Province_State'])
week5_train['Date']= pd.to_datetime(week5_train['Date']).dt.strftime("%Y%m%d").astype(int)
week5_test['Date'] = pd.to_datetime(week5_test['Date']).dt.strftime("%Y%m%d").astype(int)

In [0]:
date_wise_data = df[['Country/Region',"ObservationDate","Confirmed","Deaths","Recovered",'Active']]
date_wise_data['Date'] = date_wise_data['ObservationDate'].apply(pd.to_datetime, dayfirst=True)
date_wise_data = date_wise_data.groupby(["ObservationDate"]).sum().reset_index()
date_wise_data.rename({"ObservationDate": 'Date','Recovered':'Cured'}, axis=1,inplace= True) 
def formatted_text(string):
    display(Markdown(string))


In [0]:
# Converting columns into numberic for Train 
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
w5_X = week5_train.iloc[:,1].values
week5_train.iloc[:,1] = labelencoder.fit_transform(w5_X.astype(str))
w5_X = week5_train.iloc[:,5].values
week5_train.iloc[:,5] = labelencoder.fit_transform(w5_X)

#Converting columns into numberic Test
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
w5te_X = week5_test.iloc[:,1].values
week5_test.iloc[:,1] = labelencoder.fit_transform(w5te_X)
w5te_X = week5_test.iloc[:,5].values
week5_test.iloc[:,5] = labelencoder.fit_transform(w5te_X)

#Train & Test ======================================================
x = week5_train.iloc[:,1:6]
y = week5_train.iloc[:,6]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(x,y, test_size = 0.2, random_state = 0 )

In [0]:
#Adding Population Data
pop = pd.read_csv("/content/drive/My Drive/Master Online Exams/Dataset/population_by_country_2020.csv")
# select only population
pop = pop.iloc[:, :2]
# rename column names
pop.columns = ['Country/Region', 'Population']
# merged data
country_wise = pd.merge(country_wise, pop, on='Country/Region', how='left')
# update population
cols = ['Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Czechia', 
        'Kosovo', 'Saint Kitts and Nevis', 'Saint Vincent and the Grenadines', 
        'Taiwan*', 'US', 'West Bank and Gaza']
pops = [54409800, 89561403, 5518087, 26378274, 10708981, 1793000, 
        53109, 110854, 23806638, 330541757, 4543126]
for c, p in zip(cols, pops):
    country_wise.loc[country_wise['Country/Region']== c, 'Population'] = p
country_wise['Cases / Million People'] = round((country_wise['Confirmed'] / country_wise['Population']) * 1000000)

In [70]:
temp = country_wise.copy()
temp = temp.iloc[:,:6]
temp = temp.sort_values('Confirmed',ascending=False).reset_index()
temp.style.background_gradient(cmap='Blues',subset=["Confirmed"])\
                        .background_gradient(cmap='Reds',subset=["Deaths"])\
                        .background_gradient(cmap='Greens',subset=["Recovered"])\
                        .background_gradient(cmap='Purples',subset=["Active"])\
                        .background_gradient(cmap='PuBu',subset=["New cases"])

,index,Country/Region,Confirmed,Deaths,Recovered,Active,New cases
0,173,US,1329260,79526,216169,1033565,19710
1,157,Spain,224350,26621,136166,61563,772
2,177,United Kingdom,220449,31930,1002,187517,3924
3,85,Italy,219070,30560,105186,83324,802
4,138,Russia,209688,1915,34306,173467,11012
5,62,France,177094,26383,56327,94384,312
6,66,Germany,171879,7569,144400,19910,555
7,23,Brazil,162699,11123,64957,86619,6638
8,172,Turkey,138657,3786,92691,42180,1542
9,81,Iran,107603,6640,86143,14820,1383


In [71]:
temp = dataset.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()
temp = temp[temp['Date']==max(temp['Date'])].reset_index(drop=True)
temp1 = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Recovered'])
fig = px.pie(temp1,
             values= 'value',labels=['Active Cases','Cured','Death'],
             names="variable",
             title="Current Situation of COVID-19 in the world",
             template="seaborn")
fig.update_traces(hoverinfo='label+percent',textinfo='value', textfont_size=14,
                  marker=dict(colors=['#263fa3','#cc3c2f','#2fcc41'], line=dict(color='#FFFFFF', width=2)))
fig.update_traces(textposition='inside')
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()


In [0]:
perday2 = date_wise_data.groupby(['Date'])['Confirmed','Cured','Deaths','Active'].sum().reset_index().sort_values('Date',ascending = True)
perday2['New Daily Confirmed Cases'] = perday2['Confirmed'].sub(perday2['Confirmed'].shift())
perday2['New Daily Confirmed Cases'].iloc[0] = perday2['Confirmed'].iloc[0]
perday2['New Daily Confirmed Cases'] = perday2['New Daily Confirmed Cases'].astype(int)
perday2['New Daily Cured Cases'] = perday2['Cured'].sub(perday2['Cured'].shift())
perday2['New Daily Cured Cases'].iloc[0] = perday2['Cured'].iloc[0]
perday2['New Daily Cured Cases'] = perday2['New Daily Cured Cases'].astype(int)
perday2['New Daily Deaths Cases'] = perday2['Deaths'].sub(perday2['Deaths'].shift())
perday2['New Daily Deaths Cases'].iloc[0] = perday2['Deaths'].iloc[0]
perday2['New Daily Deaths Cases'] = perday2['New Daily Deaths Cases'].astype(int)
perday2.to_csv('/content/drive/My Drive/Master Online Exams/Dataset/perday_daily_cases.csv')

In [73]:
import plotly.express as px
fig = px.bar(perday2, x="Date", y="New Daily Confirmed Cases", barmode='group',height=500)
fig.update_layout(title_text='New COVID-19 cases reported daily all over the World',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [74]:
import plotly.express as px
fig = px.bar(perday2, x="Date", y="New Daily Cured Cases", barmode='group',height=500,
            color_discrete_sequence = ['#319146'])
fig.update_layout(title_text='New COVID-19 Recovered cases reported daily all over the world',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [0]:
# Exploratory Data Analysis

In [76]:
# 1. Displaying a sample of the train and test dataset
print('Training Dataset: \n\n', train.sample(5))
print('\n\n\n Test Dataset: \n\n', test.sample(5))

NameError: ignored

In [0]:
# Visualising the Descriptive analysis of the dataset
# 2 Visializing the data types for each colunm

print('Data Types for each columns (Training) \n\n',train.dtypes)
print('\n\n\n Data Types for each columns (Testing) \n\n',train.dtypes)

In [0]:
# 2 Visualizing the Static Information for Numerical Columns

print('Descriptiva analytics for the dataset (Training): \n\n',train.describe())
print('\n\n\n Descriptive analytics for dataset (test): \n\n' ,test.describe())

In [0]:
# Displaying some information regarding the dataset
print('Dataset information (Training): \n\n',train.info())
print('\n\n\n Dataset information (Testing): \n\n', test.info())

In [0]:
# Displaying the shape of the dataset
print('Traing dataset shape: ', train.shape)
print('\n Testing Dataset shape: ' ,test.shape)

In [0]:
# Displaying graph of the top 15 Countries
l = train[['Country_Region','TargetValue']].groupby(['Country_Region'], as_index = False).sum().sort_values(by = 'TargetValue',ascending=False)
w = pd.DataFrame(l)
data1 = l.head(15)
fig = px.bar(data1,x = 'Country_Region',y = 'TargetValue')
fig.show()

In [0]:
# pi char Vusualizing confirmed casea and faitalities
fig = px.pie(train, values='TargetValue', names='Target')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

In [0]:
# Displaing the curent world share of covid-19 cases

fig = px.pie(train, values='TargetValue', names='Country_Region')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

In [0]:
# Tree Map of Covid Cases Globaly
fig = px.treemap(train, path=['Country_Region'], values='TargetValue',
                  color='Population', hover_data=['Country_Region'],
                  color_continuous_scale='matter', title='Current share of Worldwide COVID19 Confirmed Cases')
fig.show()

In [0]:
fig = px.treemap(train, path=['Country_Region'], values='TargetValue',
                  color='Population', hover_data=['Country_Region'],
                  color_continuous_scale='matter', title='Current share of Worldwide COVID19 Confirmed Cases')
fig.show()

In [0]:
# COVID 19 Total cases growth for top 15 countries
last_date = train.Date.max()
df_countries = train[train['Date']==last_date]
df_countries = df_countries.groupby('Country_Region', as_index=False)['TargetValue'].sum()
df_countries = df_countries.nlargest(15,'TargetValue')
df_trend = train.groupby(['Date','Country_Region'], as_index=False)['TargetValue'].sum()
df_trend = df_trend.merge(df_countries, on='Country_Region')
df_trend.rename(columns={'Country_Region':'Country', 'TargetValue_x':'Cases'}, inplace=True)
px.line(df_trend, x='Date', y='Cases', color='Country', title='COVID19 Total Cases growth for top 10 worst affected countries')
